# Part 3: OTIO package exploration and debugging
## Dependencies: ffmpeg moviepy scipy librosa



In [1]:
#audio
from __future__ import print_function
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import librosa
import librosa.display

In [2]:
import pandas as pd 
import pyloudnorm as pyln

In [3]:
#video
import moviepy
from moviepy.editor import *
import opentimelineio as otio
import argparse

Failed to establish dbus connection

In [4]:
#importing audio
#x1 waveform 
#fs sampling rate

In [5]:
file_dir_list=["Data/Audio/Osi Audio Extracted.wav", "Data/Audio/Scott Audio Extracted.wav", \
               "Data/Audio/Chukwu Audio Extracted.wav", "Data/Audio/Crystal Audio Extracted.wav"]

## Testing opentimelineio


In [6]:
import opentimelineio as otio
import opentimelineio.test_utils as otio_test_utils

### First look at this overall function, then we will break down each function into its own component explanations 
```py
def main():
    args = parse_args()

    # build the structure
    tl = otio.schema.Timeline(name="Example timeline")

    tr = otio.schema.Track(name="example track")
    tl.tracks.append(tr)

    # build the clips
    for i, (fname, available_range_from_list) in enumerate(FILE_LIST):
        ref = otio.schema.ExternalReference(
            target_url=fname,
            # available range is the content available for editing
            available_range=available_range_from_list
        )

        # attach the reference to the clip
        cl = otio.schema.Clip(
            name="Clip{}".format(i + 1),
            media_reference=ref,

            # the source range represents the range of the media that is being
            # 'cut into' the clip. This is an artificial example, so its just
            # a bit shorter than the available range.
            source_range=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(
                    available_range_from_list.start_time.value + 10,
                    available_range_from_list.start_time.rate
                ),
                duration=otio.opentime.RationalTime(
                    available_range_from_list.duration.value - 20,
                    available_range_from_list.duration.rate
                ),
            )
        )

        # put the clip into the track
        tr.append(cl)

    # write the file to disk
    otio.adapters.write_to_file(tl, args.filepath)

```

In [ ]:

def _find_matching_media(name, folder):
    """Look for media with this name in this folder."""

    # This function is an example which searches the file system for matching media.
    # A real world studio implementation would likely look in an asset management system
    # and use studio-specific metadata in the clip's metadata dictionary instead
    # of matching the clip name.
    # For example:
    # shot = asset_database.find_shot(clip.metadata['mystudio']['shotID'])
    # new_media = shot.latest_render(format='mov')

    matches = glob.glob("{0}/{1}.*".format(folder, name))
    matches = list(map(os.path.abspath, matches))

    if not matches:
        # print "DEBUG: No match for clip '{0}'".format(name)
        return None
    if len(matches) == 1:
        return matches[0]
    else:
        print(
            "WARNING: {0} matches found for clip '{1}', using '{2}'".format(
                len(matches),
                name,
                matches[0]
            )
        )
        return matches[0]


### Overall summary for the layman

First create a timeline via 

## The tracks also must include 

#### Overview of the functions/variables
| Function/Variable      | Description | Components   |
| :----------       |    :----:   |          ----------: |
| FILE_LIST_INTERVALS      | A list of file locations and their intervals of time of the clip     | (fname, available_range_from_list)   |
| tl=otio.schema.Timeline(name="Example timeline")   | Text        | And more      |
| tr=otio.schema.Track(name="track")   | Defines a track         | And more      |
| tl.tracks.append(tr)  | adds track to timeline        | And more      |
| start_time=otio.opentime.RationalTime(starting sample index, sample rate)  |    This is the start time in the form of the sample index and spe     | And more      |
| duration=otio.opentime.RationalTime(400, 24)  | DURATION IS NOT THE DURATION OF THE CLIP. It is actually the ending frame followed by the sample rate       | And more      |

In [9]:
CLIP_LIST_INTERVALS= [
    # first file starts at 0 and goes 1000 frames
    #available range from list is the second element in tuple
    (
        "Data/Vidio/Batman Middle.mp4",
        otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(0, 24),
            duration=otio.opentime.RationalTime(1000, 24)
        )
    ),
    # second file starts 1001 in and goes 480 frames (at 24)
    (
        "Data/Vidio/Batman Left.mp4",
        otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(86400, 24),
            duration=otio.opentime.RationalTime(480, 24)
        )
    ),
    # third file starts at 0 and goes 400 frames @ 24)
    (
        "Data/Vidio/Batman Right.mp4",
        otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(0, 24),
            duration=otio.opentime.RationalTime(400, 24)
        )
    )
]


# clip parameters

``` py
otio.schema.Clip(
name='', 
media_reference=otio.schema.MissingReference(name='', available_range=None, metadata={}), 
source_range=None, 
metadata={})
```


In [13]:
FILE_LIST[0]

IndexError: tuple index out of range

In [ ]:
def _conform_timeline(timeline, folder):
    """ Look for replacement media for each clip in the given timeline.
    The clips are relinked in place if media with a matching name is found.
    """

    count = 0

    for clip in timeline.each_clip():
        # look for a media file that matches the clip's name
        new_path = _find_matching_media(clip.name, folder)

        # if no media is found, keep going
        if not new_path:
            continue

        # relink to the found path
        clip.media_reference = otio.schema.ExternalReference(
            target_url="file://" + new_path,
            available_range=None  # the available range is unknown without
                                  # opening the file
        )
        count += 1

    return count


## Look at this function first, then link back its functions and variables and untangle how to get a xml timeline

In [ ]:
def main():
    args = parse_args()
    #
    
    timeline = otio.adapters.read_from_file(args.input)
    count = _conform_timeline(timeline, args.folder)
    print("Relinked {0} clips to new media.".format(count))
    otio.adapters.write_to_file(timeline, args.output)
    print(
        "Saved {} with {} clips.".format(
            args.output,
            len(list(timeline.each_clip()))
        )
    )

### So to build, get the 

#### Loop the intervals, use the file number to link the camera view and the audio

In [6]:
FILE_LIST= [
    # first file starts at 0 and goes 1000 frames
    (
        "Batman Middle.mp4",
        otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(0, 24),
            duration=otio.opentime.RationalTime(1000, 24)
        )
    ),
    # second file starts 1001 in and goes 480 frames (at 24)
    (
        "Batman Left.mp4",
        otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(8640, 24),
            duration=otio.opentime.RationalTime(480, 24)
        )
    ),
    # third file starts at 0 and goes 400 frames @ 24)
    (
        "Batman Right.mp4",
        otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(0, 24),
            duration=otio.opentime.RationalTime(400, 24)
        )
    ),
    (
        "Batman Middle.mp4",
        otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(4000, 24),
            duration=otio.opentime.RationalTime(2400, 24)
        )
    )
]


In [4]:
FILE_LIST= [
    # first file starts at 0 and goes 1000 frames
    (
        "Batman Middle.mp4",
        otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(0, 24),
            duration=otio.opentime.RationalTime(1000, 24)
        )
    )
]


In [7]:

# build the structure
tl = otio.schema.Timeline(name="Example timeline")

tr = otio.schema.Track(name="example track")
tl.tracks.append(tr)

# build the clips
for i, (fname, available_range_from_list) in enumerate(FILE_LIST):
    ref = otio.schema.ExternalReference(
        target_url=fname,
        # available range is the content available for editing
        available_range=available_range_from_list
    )

    # attach the reference to the clip
    cl = otio.schema.Clip(
        name="Clip{}".format(i + 1),
        media_reference=ref,

        # the source range represents the range of the media that is being
        # 'cut into' the clip. This is an artificial example, so its just
        # a bit shorter than the available range.
        source_range=otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(
                available_range_from_list.start_time.value + 10,
                available_range_from_list.start_time.rate
            ),
            duration=otio.opentime.RationalTime(
                available_range_from_list.duration.value - 20,
                available_range_from_list.duration.rate
            ),
        )
    )

    # put the clip into the track
    tr.append(cl)

# write the file to disk
otio.adapters.write_to_file(tl, 'test.otio')

True

# Testing format of xml and OTIO 
### first convert kdenlive project file to xml and otio then see if it can be imported back in

In [13]:
kdsm=otio.adapters.read_from_file('testing.kdenlive')

### Conversion from kdenlive to otio works when importing back to kdenlive

In [14]:
otio.adapters.write_to_file(kdsm, "kd_conv.otio")

True

### conversion from kdenlive to xml (using otio package) then importing back to kdenlive does not work


In [15]:
otio.adapters.write_to_file(kdsm, "kd_conv.xml")

'kd_conv.xml'


## The next step is to see if the kdenlive ->otio can be converted into an acceptable readable xml form
# kdenlive ->otio -> xml does not work, the issue is reading the xml from 

In [21]:
kd_otio_xml_test=otio.adapters.read_from_file('kdtestimport.kdenlive')
otio.adapters.write_to_file(kd_otio_xml_test, "kd_otio_xml_conv.xml")

'kd_otio_xml_conv.xml'

In [26]:
kd_otio_xml_test=otio.adapters.otio("kd_conv.otio")

AttributeError: module 'opentimelineio.adapters' has no attribute 'otio'

# See how a correct otio and xml form works and try to match the form


# Does xml work at all?

# It DOES


# now we translate the xml to otio then see

In [4]:
xml_to_otio_test=otio.adapters.read_from_file('2nd_Example_Premiere.xml')
otio.adapters.write_to_file(xml_to_otio_test, "scott_example.otio")

True

In [5]:
xml_to_otio_test

otio.schema.Timeline(name='Synced Sequence', tracks=otio.schema.Stack(name='Synced Sequence', children=[otio.schema.Track(name='', children=[otio.schema.Gap(name='', source_range=otio.opentime.TimeRange(start_time=otio.opentime.RationalTime(value=0, rate=29.97), duration=otio.opentime.RationalTime(value=743, rate=29.97)), effects=[], markers=[], metadata={}), otio.schema.Clip(name='White Side.MP4', media_reference=otio.schema.ExternalReference(target_url='file://localhost/Volumes/Extreme%20SSD/_S%20Projects/Dreamlet/Products/Jeff/2nd%20XML/White%20Side.MP4'), source_range=otio.opentime.TimeRange(start_time=otio.opentime.RationalTime(value=0, rate=29.97), duration=otio.opentime.RationalTime(value=4500, rate=29.97)), metadata={'fcp_xml': {'@id': 'clip2', '@premiereChannelType': 'stereo', 'link': [{'clipindex': '1', 'groupindex': '1', 'linkclipref': 'clip2', 'mediatype': 'video', 'trackindex': '1'}, {'clipindex': '1', 'groupindex': '1', 'linkclipref': 'A10E14C4-D6DA-4DA8-87A9-F7B80F96F122

In [ ]:
Timeline(name='Synced Sequence', 
         tracks=otio.schema.Stack( 
                                    name='Synced Sequence', 
                                    children=[otio.schema.Track(name='',
                    children=[otio.schema.Gap(name='', source_range=otio.opentime.TimeRange(start_time=otio.opentime.RationalTime(value=0, rate=29.97), duration=otio.opentime.RationalTime(value=743, rate=29.97)), effects=[], markers=[], metadata={})

In [ ]:
Timeline(
        name='Synced Sequence', 
        tracks=otio.schema.Stack(
            name='Synced Sequence', 
            children=[otio.schema.Track()